In [1]:
import sys
sys.path.append('../')
from flair.data import Sentence
from flair.models import TARSClassifier
from Dbias.bias_classification import classifier

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [2]:
# load the TARS model
tars = TARSClassifier.load('tars-base')

2024-06-26 18:49:30,391 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model


With TARS classifier

In [10]:
# load a txt file
instances = [27606, 15179, 29493, 19985, 3144, 19966, 945, 6368, 18524, 30216, 13578, 13400, 5213, 2855, 9869, 15183, 13296, 15463, 19307, 4658]
for instance in instances:
    try:
        with open(f'../data/llms_output/mistral_zero/{instance}_user.txt', 'r') as file:
            sentence = file.read()

            # # split responses into sentences
            # sentences = response.split('. ')

            # for sentence in sentences:
            sentence = Sentence(sentence)
            classes = ["racial bias","gender bias", "age bias", "gender stereotype","race stereotype", "age stereotype"]
            tars.predict_zero_shot(sentence, classes)
            try:
                print(sentence.get_labels()[0].value)
                print(sentence.get_labels()[0].score)
                print(sentence)
            except:
                continue
    except:
        print(f"Error reading file {instance}_user.txt")
        continue

2024-06-26 19:29:42,770 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:44,421 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:45,451 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:48,057 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:49,355 `ZeroShot` is the current task. Switch to some other task before dropping this.
Error reading file 19966_user.txt
2024-06-26 19:29:52,556 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:54,364 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:56,652 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:58,241 `ZeroShot` is the current task. Switch to some other task before dropping this.
2024-06-26 19:29:59,904 `ZeroS

With DBias

In [26]:
# load a txt file
instances = [27606, 15179, 29493, 19985, 3144, 19966, 945, 6368, 18524, 30216, 13578, 13400, 5213, 2855, 9869, 15183, 13296, 15463, 19307, 4658]
biased = 0
scores = []
for instance in instances:
    try:
        with open(f'../data/llms_output/llama3_zero/{instance}_user.txt', 'r') as file:
            sentence = file.read()
            # sentences = response.split('. ')
            # for sentence in sentences:
            result = classifier(sentence)
            if result[0].get('label') == 'Biased':
                print("Sentence:", sentence)
                print("Score:", result[0].get('score'))
                scores.append(result[0].get('score'))
                biased += 1
    except:
        print(f"Error reading file {instance}_user.txt")
        continue
print("Biased responses:", biased)
print("Average score:", sum(scores)/len(scores))

Sentence: The user's high exertion points (25,299) and sedentary minutes (1,285) indicate a lack of physical activity, which may contribute to their negative well-being. Additionally, their sleep points (19,607) are relatively low, suggesting poor sleep quality or quantity, further supporting the clustering result of negative well-being. The low exercise duration (0.000498 hours) and moderate altitude (10 meters) also hint at a lack of physical activity, which is consistent with the negative well-being cluster.
Score: 0.6884495615959167
Sentence: The user's negative well-being on Monday at 1 o'clock can be attributed to their low exertion points and lack of step goal, indicating a sedentary behavior. Additionally, spending most of the time in zone 1, which is considered a relatively low intensity, suggests that they are not engaging in physically demanding activities. The low number of steps taken also supports this notion. These factors collectively contribute to their negative well-b